In [ ]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")
    


In [ ]:
%pip install torch torchvision pandas python-dotenv kaggle scikit-learn

In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split

In [3]:
def download_data_from_kaggle():
    """
    Downloads the dataset from Kaggle and unzips it.
    Assumes environment variables KAGGLE_USERNAME and KAGGLE_KEY are set.
    """
    # Colab-specific: only use these if running on Google Colab
    try:
        from google.colab import userdata
        os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
        os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
    except ImportError:
        # load .env
        from dotenv import load_dotenv
        load_dotenv()

    # Download via Kaggle API
    os.system("kaggle competitions download -c histopathologic-cancer-detection")
    os.system("unzip -nq histopathologic-cancer-detection.zip")

In [ ]:
download_data_from_kaggle()

In [21]:
train_dir = 'train'
test_dir = 'test'
csv_path = 'train_labels.csv'

In [31]:
# Load labels
labels = pd.read_csv(csv_path)

# Split into training and validation sets
train_labels, val_labels = train_test_split(labels, test_size=0.2, random_state=42)

In [49]:
# import ImageDataset as ImageDataset # on MacOS, import ImageDataset
class ImageDataset(Dataset):
    def __init__(self, labels, image_folder, transform=None):
        self.labels = labels
        self.image_folder = image_folder
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get image path and label
        img_name = os.path.join(self.image_folder, self.labels.iloc[idx, 0])
        label = self.labels.iloc[idx, 1]

        # Open and transform image
        image = Image.open(img_name + ".tif").convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Convert label to tensor
        label = torch.tensor(label, dtype=torch.float32)
        return image, label


In [50]:
class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        """
        EarlyStopping class to monitor the validation loss and stop if no improvement for 'patience' epochs.

        Args:
            patience (int): How many epochs to wait after the last time validation loss improved.
            delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.best_model_wts = None

    def __call__(self, val_loss, model):
        """
        Call method to check if the validation loss improved. If not, increase the counter.

        Args:
            val_loss (float): The current validation loss.
            model (torch.nn.Module): The model being trained.
        """
        score = -val_loss  # We want to minimize the validation loss

        if self.best_score is None:
            self.best_score = score
            self.best_model_wts = model.state_dict()
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.best_model_wts = model.state_dict()
            self.counter = 0

    def get_best_weights(self):
        """
        Returns the weights of the best model found so far.
        """
        return self.best_model_wts


In [ ]:
# Define transformations for normalization, ResNet requires
# resizing to at least 224 W and H
transform = transforms.Compose([
    transforms.Resize((96,96)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
])

predict_transform = transforms.Compose([
    transforms.Resize((96,96)),
    transforms.ToTensor(),
])


# Create datasets
train_dataset = ImageDataset(train_labels, train_dir, transform=transform)
val_dataset = ImageDataset(val_labels, train_dir, transform=predict_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=0, pin_memory=True)


In [ ]:
class ConvNet96x96(nn.Module):
    def __init__(self, num_classes=1):
        super(ConvNet96x96, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1) # 96x96x32
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1) # 96x96x64
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1) # 96x96x128

        # Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Reduces each spatial dimension by half

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 24 * 24, 512) # Corrected size
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, num_classes)

        # Dropout (to prevent overfitting)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Convolutional layers with ReLU and pooling
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten the tensor
        x = x.view(x.size(0), -1)

        # Fully connected layers with ReLU and dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)

        return x

model = ConvNet96x96()

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = model.to(device)

print(model)
print(device)

In [53]:
# Binary Cross-Entropy Loss for binary classification
criterion = nn.BCEWithLogitsLoss()

# SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
num_epochs = 100

early_stopping = EarlyStopping(patience=5, delta=0.001)

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    model.train()
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        print(f"Batch {i+1}/{len(train_loader)}", end="\r")
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Training Loss: {running_loss / len(train_loader)}")
    print("Validation:")
    model.eval()
    val_loss = 0.0
    correct = 0

    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs.squeeze(), labels).item()
            predicted = (outputs.squeeze() > 0.5).long()
            correct += (predicted == labels).sum().item()
            print(f"Validation Loss: {val_loss / len(val_loader)}")
    print(f"Validation Loss: {val_loss / len(val_loader)}")
    print(f"Validation Accuracy: {correct / len(val_dataset)}")

    # Early stopping logic
    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# Load the best model weights
model.load_state_dict(early_stopping.get_best_weights())



In [ ]:
torch.save(model.state_dict(), model_name + '.pth')


In [ ]:
model.load_state_dict(torch.load(model_name + '.pth'))
model.eval()

def predict_image(image_path, model, transform, device):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    output = model(image).item()
    return '1' if output > 0.5 else '0'


In [ ]:
# for each image in test dir, predict and produce CSV with results
import os
import pandas as pd
from PIL import Image

results = []

for filename in os.listdir(test_dir):
    if filename.endswith('.tif'):
        image_path = os.path.join(test_dir, filename)
        prediction = predict_image(image_path, model, predict_transform, device)
        id_no_ext = os.path.splitext(filename)[0]
        results.append({'id': id_no_ext, 'label': prediction})

results = pd.DataFrame(results)

results.to_csv(model_name + '.csv', index=False)

In [ ]:
!kaggle competitions submit -c histopathologic-cancer-detection -f {model_name + '.csv'} -m {model_name}